In [1]:
import pandas as pd

Importamos el archivo business anterior 

In [2]:
df_business_yelp = pd.read_csv('dataset_business.csv')
df_business_yelp.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN


Traamos unicamente las palabras que nos interesa de la columna categories que nos interesan

In [3]:
#Crear una copia del DataFrame original
df_temp = df_business_yelp.copy()

#Eliminar filas con valores NaN en la columna 'categories'
df_temp = df_temp.dropna(subset=['categories'])

#Lista de palabras clave
nombres = ['Restaurant', 'Fast food restaurant', 'Pizza restaurant', 'Pizza Takeout', 'Takeout Restaurant', 'Delivery Restaurant', 'Mexican restaurant', 'American restaurant', 'Burger restaurant', 'Family restaurant', 'Sandwich shop', 'Deli', 'Italian restaurant', 'Chinese restaurant', 'Coffee shop', 'Cafe', 'Bakery', 'Seafood restaurant', 'Barbecue restaurant', 'Asian restaurant', 'Sushi restaurant', 'Japanese restaurant', 'Thai restaurant', 'Chicken restaurant', 'Chicken wings restaurant', 'Bakery', 'Bakery shop', 'Ice cream shop', 'Indian restaurant', 'Latin American restaurant', 'Juice shop', 'Caterer', 'Taco restaurant', 'Vegetarian restaurant']

#Filtrar el DataFrame
df_filtrado = df_temp[df_temp['categories'].str.contains('|'.join(nombres))]
#Reseteo los índices de las columnas
df_filtrado = df_filtrado.reset_index(drop=True)


In [4]:
df_filtrado.head(3)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN


In [5]:
import geopandas as gpd
import os

Importamos el archivo de geometry florida

In [6]:
os.environ['SHAPE_RESTORE_SHX'] = 'YES'
df_geometry = gpd.read_file('FL_BOUNDARY_DETAILED.shp')
df_geometry.head()
df_geometry.dtypes

geometry    geometry
dtype: object

Concatenamos el dataset business que estamos trabajando con el archivo geometry

In [7]:
df_concat = pd.concat([df_geometry, df_filtrado,], axis=1)
df_concat.head(2)

,geometry,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,"MULTIPOLYGON (((-85.54423 30.00015, -85.54407 ...",MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,None,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."


Creamos al funcion que se va a encargar de decirsnos que filas estan en Florida

In [8]:
from shapely.geometry import Point, MultiPolygon

def encontrar_ubicaciones(dataframe, geometry_column='geometry'):
    # Obtener el polígono del DataFrame
    multipoligono = dataframe[geometry_column].iloc[0]
    
    # Crear un objeto para el polígono Shapely
    multi_polygon = MultiPolygon(multipoligono.geoms)
    
    # Función para verificar si está a ubicación  dentro del multipolígono
    def esta_dentro(latitud, longitud):
        punto = Point(longitud, latitud)
        return multi_polygon.contains(punto)
    
    # Aplicar la función a todas las filas y obtener una serie de True/False
    dataframe['ubicacion'] = dataframe.apply(lambda row: esta_dentro(row['latitude'], row['longitude']), axis=1)
    
    # Organizar el DataFrame por la columna 'ubicacion' primero los True
    dataframe.sort_values(by='ubicacion', ascending=False, inplace=True)
    
    return dataframe

In [9]:
#Tiempo aproximado 30 minutos
resultados = encontrar_ubicaciones(df_concat)

In [10]:
resultados.head(3)

,geometry,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,ubicacion
14548,None,k5LRHoQHGyaLjC9i6RkGcQ,Subway,3208 Lithia-pinecrest Rd,Valrico,NV,33596,27.896959,-82.247303,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Sandwiches, Restaurants, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",True
45310,None,H6tPRvGHvm-TdQ40LZE0aw,Dragon Express,2101 E Fletcher Ave,Tampa,PA,33612,28.068850,-82.435273,4.0,22,1,"{'RestaurantsTakeOut': 'True', 'BusinessAccept...","American (New), Chinese, Restaurants","{'Monday': '11:0-22:30', 'Wednesday': '11:0-22...",True
45306,None,h7pY4--PXa5GvZaskzAy-g,Rodie's Restaurant and Pancake House,1097 S Pinellas Ave,Tarpon Springs,FL,34689,28.138015,-82.756277,4.0,102,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Breakfast & Brunch, Restaurants, American (Tra...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",True


In [11]:
resultados.shape

(53399, 16)

Filtramos el artchivo unicamente donde la columna ubicación se True lo cual indica que esta en Florida

In [12]:
datos_florida = resultados[resultados['ubicacion'] == True]
datos_florida.head(5)

,geometry,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,ubicacion
14548,None,k5LRHoQHGyaLjC9i6RkGcQ,Subway,3208 Lithia-pinecrest Rd,Valrico,NV,33596,27.896959,-82.247303,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Sandwiches, Restaurants, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",True
45310,None,H6tPRvGHvm-TdQ40LZE0aw,Dragon Express,2101 E Fletcher Ave,Tampa,PA,33612,28.068850,-82.435273,4.0,22,1,"{'RestaurantsTakeOut': 'True', 'BusinessAccept...","American (New), Chinese, Restaurants","{'Monday': '11:0-22:30', 'Wednesday': '11:0-22...",True
45306,None,h7pY4--PXa5GvZaskzAy-g,Rodie's Restaurant and Pancake House,1097 S Pinellas Ave,Tarpon Springs,FL,34689,28.138015,-82.756277,4.0,102,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Breakfast & Brunch, Restaurants, American (Tra...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",True
45304,None,CX7eMbe7_EQEh_dk4eBxYg,The Blue Dolphin Restaurant,8825 State Rd 52,Hudson,TN,34667,28.331650,-82.669045,3.0,9,0,"{'NoiseLevel': ""u'quiet'"", 'HasTV': 'False', '...","Restaurants, Seafood, Steakhouses, Italian","{'Tuesday': '11:30-21:0', 'Wednesday': '11:30-...",True
45302,None,8fIIeyRpL-osIWF8oDGOeA,Panda Express,7000 Us Highway 19 N,Pinellas Park,FL,33781,27.835905,-82.682159,3.0,56,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Chinese, Fast Food, Restaurants","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",True


Observamos la frencuencia de las palabras filtradas en la columna categories al inicio

In [13]:
# Crear un diccionario para almacenar el recuento de cada palabra clave
conteo_palabras = {}

# Iterar sobre cada palabra clave en la lista nombres
for palabra in nombres:
    # Contar el número de veces que aparece la palabra clave en la columna 'categories'
    conteo_palabras[palabra] = datos_florida['categories'].str.count(palabra).sum()

# Mostrar el recuento de cada palabra clave
for palabra, conteo in conteo_palabras.items():
    print(f'{palabra}: {conteo}')

Restaurant: 8710
Fast food restaurant: 0
Pizza restaurant: 0
Pizza Takeout: 0
Takeout Restaurant: 0
Delivery Restaurant: 0
Mexican restaurant: 0
American restaurant: 0
Burger restaurant: 0
Family restaurant: 0
Sandwich shop: 0
Deli: 598
Italian restaurant: 0
Chinese restaurant: 0
Coffee shop: 0
Cafe: 593
Bakery: 0
Seafood restaurant: 0
Barbecue restaurant: 0
Asian restaurant: 0
Sushi restaurant: 0
Japanese restaurant: 0
Thai restaurant: 0
Chicken restaurant: 0
Chicken wings restaurant: 0
Bakery shop: 0
Ice cream shop: 0
Indian restaurant: 0
Latin American restaurant: 0
Juice shop: 0
Caterer: 341
Taco restaurant: 0
Vegetarian restaurant: 0


Creamos una copia para su trabajo

In [14]:
df_prueba = datos_florida

Creamos una función que se encargue de clasifica esas palabras en una unica categoria y la asigne a una columna 'business_category'

In [16]:
def assign_business_category(df):
    # Crea la columna 'business_category' si no existe y rellénala con 'Restaurant'
    if 'business_category' not in df:
        df['business_category'] = 'Restaurant'
    else:
        df['business_category'].fillna('Restaurant', inplace=True)

    # Define las reglas de asignación de categorías
    category_rules = {
        'Fast food|Pizza': 'Fast Food',
        'Takeout|Delivery': 'Takeout/Delivery',
        'Mexican|Taco': 'Mexican Cuisine',
        'American|Burger|Family': 'American Cuisine',
        'Sandwich|Deli': 'Sandwich Shop',
        'Italian': 'Italian Cuisine',
        'Chinese': 'Chinese Cuisine',
        'Coffee|Bakery': 'Coffee Shop',
        'Seafood': 'Seafood Restaurant',
        'Barbecue': 'Barbecue Restaurant',
        'Asian|Sushi|Japanese|Thai': 'Asian Cuisine',
        'Chicken|Wing|Wings': 'Chicken Restaurant',
        'Bakery': 'Bakery',
        'Ice|cream': 'Ice Cream Shop',
        'Indian': 'Indian Cuisine',
        'Latin': 'Latin American Cuisine',
        'Juice': 'Juice Shop',
        'Caterer': 'Specialty Dining',
        'Vegetarian': 'Vegetarian Restaurant',
        'Vietnamese': 'Vietnamese Restaurant'
    }

    # Aplica las reglas para asignar las categorías
    for pattern, category in category_rules.items():
        df.loc[df['categories'].str.contains(pattern, case=False), 'business_category'] = category
    
    return df

In [17]:
assign_business_category(df_prueba)

C:\Users\Jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,ubicacion,business_category
14548,None,k5LRHoQHGyaLjC9i6RkGcQ,Subway,3208 Lithia-pinecrest Rd,Valrico,NV,33596,27.896959,-82.247303,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Sandwiches, Restaurants, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",True,Sandwich Shop
45310,None,H6tPRvGHvm-TdQ40LZE0aw,Dragon Express,2101 E Fletcher Ave,Tampa,PA,33612,28.068850,-82.435273,4.0,22,1,"{'RestaurantsTakeOut': 'True', 'BusinessAccept...","American (New), Chinese, Restaurants","{'Monday': '11:0-22:30', 'Wednesday': '11:0-22...",True,Chinese Cuisine
45306,None,h7pY4--PXa5GvZaskzAy-g,Rodie's Restaurant and Pancake House,1097 S Pinellas Ave,Tarpon Springs,FL,34689,28.138015,-82.756277,4.0,102,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Breakfast & Brunch, Restaurants, American (Tra...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",True,American Cuisine
45304,None,CX7eMbe7_EQEh_dk4eBxYg,The Blue Dolphin Restaurant,8825 State Rd 52,Hudson,TN,34667,28.331650,-82.669045,3.0,9,0,"{'NoiseLevel': ""u'quiet'"", 'HasTV': 'False', '...","Restaurants, Seafood, Steakhouses, Italian","{'Tuesday': '11:30-21:0', 'Wednesday': '11:30-...",True,Seafood Restaurant
45302,None,8fIIeyRpL-osIWF8oDGOeA,Panda Express,7000 Us Highway 19 N,Pinellas Park,FL,33781,27.835905,-82.682159,3.0,56,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Chinese, Fast Food, Restaurants","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",True,Chinese Cuisine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,None,WcUS0B2iFLPAsdUZMwKkWw,Surf and Turf Cafe,101 N Franklin St,Tampa,PA,33602,27.945657,-82.456744,3.0,5,0,"{'Ambience': ""{'touristy': False, 'hipster': F...","Seafood, Restaurants, Sandwiches","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",True,Seafood Restaurant
32079,None,ykJ9zt1fJ1cOpxz5FkpVOQ,I C Sharks Waterfront Cafe & Tiki Bar,13040 Gandy Blvd N,St Petersburg,AZ,33702,27.870841,-82.612050,4.0,186,0,"{'RestaurantsTakeOut': 'True', 'Caters': 'True...","Seafood, Specialty Food, Active Life, Seafood ...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ...",True,Seafood Restaurant
49106,None,fM8ZCk2SlMu--joYJQ3EQA,Starbucks,6901 22nd Ave N,St. Petersburg,PA,33710,27.793570,-82.735540,3.0,15,0,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Cafes, Coffee & Tea, Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '10:30-18:0',...",True,Coffee Shop
51065,None,bm-Hrhg0smyGTsSNi-5ngw,Tropical Smoothie Cafe,1840 Gulf To Bay Blvd,Clearwater,CA,33765,27.960900,-82.761113,2.0,41,1,"{'RestaurantsReservations': 'False', 'Business...","Juice Bars & Smoothies, Sandwiches, Food, Rest...","{'Monday': '6:30-23:0', 'Tuesday': '6:30-23:0'...",True,Juice Shop


Observamos que no tengamos valores diferentes a los establecidos anteriormente en neustra columna 'business_category'

In [18]:
df_prueba['business_category'].unique()

array(['Sandwich Shop', 'Chinese Cuisine', 'American Cuisine',
       'Seafood Restaurant', 'Specialty Dining', 'Restaurant',
       'Chicken Restaurant', 'Vietnamese Restaurant', 'Mexican Cuisine',
       'Coffee Shop', 'Italian Cuisine', 'Ice Cream Shop', 'Juice Shop',
       'Asian Cuisine', 'Vegetarian Restaurant', 'Indian Cuisine',
       'Fast Food', 'Latin American Cuisine'], dtype=object)

Comprobamos una vez la  la frencuencia en este caso ya no de las palabras sino de las categorias creadas 

In [19]:
df_prueba['business_category'].value_counts()

business_category
American Cuisine          1406
Restaurant                1067
Sandwich Shop              818
Seafood Restaurant         643
Asian Cuisine              614
Chicken Restaurant         601
Ice Cream Shop             495
Coffee Shop                490
Mexican Cuisine            487
Italian Cuisine            472
Fast Food                  333
Specialty Dining           329
Chinese Cuisine            262
Latin American Cuisine     254
Juice Shop                 195
Vegetarian Restaurant      169
Vietnamese Restaurant      112
Indian Cuisine              82
Name: count, dtype: int64

Crear df para los id_category

In [36]:
# Definir los valores
categories = ['Fast Food', 'Takeout/Delivery', 'Mexican Cuisine', 'American Cuisine', 'Sandwich Shop',
              'Italian Cuisine', 'Chinese Cuisine', 'Coffee Shop', 'Seafood Restaurant', 'Barbecue Restaurant',
              'Asian Cuisine', 'Chicken Restaurant', 'Bakery', 'Ice Cream Shop', 'Indian Cuisine',
              'Latin American Cuisine', 'Juice Shop', 'Specialty Dining', 'Vegetarian Restaurant',
              'Vietnamese Restaurant', 'Restaurant']
# Crear el DataFrame

df_categories_id = pd.DataFrame({'business_category': categories})


In [37]:
# Generar los category_id
import base64
df_categories_id['category_id'] = df_categories_id['business_category'].apply(lambda x: base64.b64encode(str(hash(x)).encode()).decode()[:5])
df_categories_id

,business_category,category_id
0,Fast Food,LTQyM
1,Takeout/Delivery,NzYwO
2,Mexican Cuisine,NzM0N
3,American Cuisine,OTE3M
4,Sandwich Shop,Nzk4O
5,Italian Cuisine,LTQ4M
6,Chinese Cuisine,ODU1N
7,Coffee Shop,Nzk0M
8,Seafood Restaurant,LTg5M
9,Barbecue Restaurant,LTExO


Revisamos Duplicados

In [35]:
duplicates = df_categories_id[df_categories_id.duplicated('category_id', keep=False)]

if not duplicates.empty:
    print("¡Se encontraron category_id duplicados!")
    print(duplicates)
else:
    print("No hay category_id duplicados.")

No hay category_id duplicados.


In [46]:
df_categories_id.to_csv('dataset_category_id.csv', index=False)

In [38]:
df_prueba1 = df_prueba.merge(df_categories_id[['business_category', 'category_id']], on='business_category', how='left')
df_prueba1

,geometry,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,ubicacion,business_category,category_id
0,None,k5LRHoQHGyaLjC9i6RkGcQ,Subway,3208 Lithia-pinecrest Rd,Valrico,NV,33596,27.896959,-82.247303,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Sandwiches, Restaurants, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",True,Sandwich Shop,Nzk4O
1,None,H6tPRvGHvm-TdQ40LZE0aw,Dragon Express,2101 E Fletcher Ave,Tampa,PA,33612,28.068850,-82.435273,4.0,22,1,"{'RestaurantsTakeOut': 'True', 'BusinessAccept...","American (New), Chinese, Restaurants","{'Monday': '11:0-22:30', 'Wednesday': '11:0-22...",True,Chinese Cuisine,ODU1N
2,None,h7pY4--PXa5GvZaskzAy-g,Rodie's Restaurant and Pancake House,1097 S Pinellas Ave,Tarpon Springs,FL,34689,28.138015,-82.756277,4.0,102,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Breakfast & Brunch, Restaurants, American (Tra...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",True,American Cuisine,OTE3M
3,None,CX7eMbe7_EQEh_dk4eBxYg,The Blue Dolphin Restaurant,8825 State Rd 52,Hudson,TN,34667,28.331650,-82.669045,3.0,9,0,"{'NoiseLevel': ""u'quiet'"", 'HasTV': 'False', '...","Restaurants, Seafood, Steakhouses, Italian","{'Tuesday': '11:30-21:0', 'Wednesday': '11:30-...",True,Seafood Restaurant,LTg5M
4,None,8fIIeyRpL-osIWF8oDGOeA,Panda Express,7000 Us Highway 19 N,Pinellas Park,FL,33781,27.835905,-82.682159,3.0,56,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Chinese, Fast Food, Restaurants","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",True,Chinese Cuisine,ODU1N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8824,None,WcUS0B2iFLPAsdUZMwKkWw,Surf and Turf Cafe,101 N Franklin St,Tampa,PA,33602,27.945657,-82.456744,3.0,5,0,"{'Ambience': ""{'touristy': False, 'hipster': F...","Seafood, Restaurants, Sandwiches","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",True,Seafood Restaurant,LTg5M
8825,None,ykJ9zt1fJ1cOpxz5FkpVOQ,I C Sharks Waterfront Cafe & Tiki Bar,13040 Gandy Blvd N,St Petersburg,AZ,33702,27.870841,-82.612050,4.0,186,0,"{'RestaurantsTakeOut': 'True', 'Caters': 'True...","Seafood, Specialty Food, Active Life, Seafood ...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ...",True,Seafood Restaurant,LTg5M
8826,None,fM8ZCk2SlMu--joYJQ3EQA,Starbucks,6901 22nd Ave N,St. Petersburg,PA,33710,27.793570,-82.735540,3.0,15,0,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Cafes, Coffee & Tea, Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '10:30-18:0',...",True,Coffee Shop,Nzk0M
8827,None,bm-Hrhg0smyGTsSNi-5ngw,Tropical Smoothie Cafe,1840 Gulf To Bay Blvd,Clearwater,CA,33765,27.960900,-82.761113,2.0,41,1,"{'RestaurantsReservations': 'False', 'Business...","Juice Bars & Smoothies, Sandwiches, Food, Rest...","{'Monday': '6:30-23:0', 'Tuesday': '6:30-23:0'...",True,Juice Shop,NDIwM


Seleccionamos las columnas finales del dataset

In [54]:
df_prueba1 = df_prueba1[['business_id','name','category_id','business_category','ubicacion', 'postal_code','latitude','longitude','stars','review_count','hours','address','city','attributes']]
df_prueba1.head()

,business_id,name,category_id,business_category,ubicacion,postal_code,latitude,longitude,stars,review_count,hours,address,city,attributes
0,k5LRHoQHGyaLjC9i6RkGcQ,Subway,Nzk4O,Sandwich Shop,True,33596,27.896959,-82.247303,3.5,8,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",3208 Lithia-pinecrest Rd,Valrico,"{'RestaurantsPriceRange2': '1', 'GoodForKids':..."
1,H6tPRvGHvm-TdQ40LZE0aw,Dragon Express,ODU1N,Chinese Cuisine,True,33612,28.068850,-82.435273,4.0,22,"{'Monday': '11:0-22:30', 'Wednesday': '11:0-22...",2101 E Fletcher Ave,Tampa,"{'RestaurantsTakeOut': 'True', 'BusinessAccept..."
2,h7pY4--PXa5GvZaskzAy-g,Rodie's Restaurant and Pancake House,OTE3M,American Cuisine,True,34689,28.138015,-82.756277,4.0,102,"{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",1097 S Pinellas Ave,Tarpon Springs,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa..."
3,CX7eMbe7_EQEh_dk4eBxYg,The Blue Dolphin Restaurant,LTg5M,Seafood Restaurant,True,34667,28.331650,-82.669045,3.0,9,"{'Tuesday': '11:30-21:0', 'Wednesday': '11:30-...",8825 State Rd 52,Hudson,"{'NoiseLevel': ""u'quiet'"", 'HasTV': 'False', '..."
4,8fIIeyRpL-osIWF8oDGOeA,Panda Express,ODU1N,Chinese Cuisine,True,33781,27.835905,-82.682159,3.0,56,"{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",7000 Us Highway 19 N,Pinellas Park,"{'RestaurantsGoodForGroups': 'True', 'Restaura..."


Exportamos el dataset terminado

In [56]:
df_prueba1.shape

(8829, 14)

In [57]:
df_prueba1.isnull().sum()

business_id            0
name                   0
category_id            0
business_category      0
ubicacion              0
postal_code            3
latitude               0
longitude              0
stars                  0
review_count           0
hours                984
address              113
city                   0
attributes           111
dtype: int64

In [55]:
df_prueba1.to_csv('dataset_business_final_yelp.csv', index=False)